# This code converts DOTA dataset obb format to YOLO obb format

In [7]:
import os
from PIL import Image

# Class mapping
class_mapping = {
    "plane": 0,
    "ship": 1,
    "harbor": 2,
    "bridge": 3,
    "large-vehicle": 4,
    "small-vehicle": 5,
    "helicopter": 6,
    "swimming-pool": 7,
}

# Folders
image_folder = "val_images"
label_folder = "val_labels"
output_folder = "obb_labels"

os.makedirs(output_folder, exist_ok=True)

# Process files
for image_name in os.listdir(image_folder):
    if image_name.endswith((".jpg", ".png", ".jpeg")):
        # Get image dimensions
        image_path = os.path.join(image_folder, image_name)
        img = Image.open(image_path)
        width, height = img.size

        # Corresponding label file
        label_name = os.path.splitext(image_name)[0] + ".txt"
        label_path = os.path.join(label_folder, label_name)

        if os.path.exists(label_path):
            with open(label_path, "r") as file:
                lines = file.readlines()[2:]  # Skip the first two lines

            output_lines = []
            for line in lines:
                parts = line.strip().split()
                coords = list(map(float, parts[:8]))  # Get 8 coordinates
                class_name = parts[8]  # Class name

                # Map class name to index
                class_index = class_mapping.get(class_name, -1)
                if class_index == -1:
                    continue  # Skip unknown classes

                # Normalize coordinates
                normalized_coords = [coords[i] / width if i % 2 == 0 else coords[i] / height for i in range(8)]

                # Format output
                output_line = f"{class_index} " + " ".join(f"{coord:.6f}" for coord in normalized_coords)
                output_lines.append(output_line)

            # Save to output file
            output_path = os.path.join(output_folder, label_name)
            with open(output_path, "w") as out_file:
                out_file.write("\n".join(output_lines))


# This code crops images and label in YOLO OBB format

In [2]:
# import os
# import cv2
# import numpy as np

# # Define input and output directories
# image_dir = 'training_images'  # Path to your images
# label_dir = 'obb_labels'       # Path to your YOLO OBB labels
# output_image_dir = 'cropped_images'  # Path to save cropped images
# output_label_dir = 'cropped_labels'  # Path to save cropped labels

# # Create output directories if they don't exist
# os.makedirs(output_image_dir, exist_ok=True)
# os.makedirs(output_label_dir, exist_ok=True)

# # Define the crop size (1024x1024)
# crop_size = 1024

# # Get all image and label files
# image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
# label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

# # Process the images without progress bar
# for image_file, label_file in zip(image_files, label_files):
#     image_path = os.path.join(image_dir, image_file)
#     label_path = os.path.join(label_dir, label_file)

#     # Read the image
#     image = cv2.imread(image_path)
#     h, w = image.shape[:2]

#     # Read the labels
#     with open(label_path, 'r') as f:
#         labels = f.readlines()

#     cropped_count = 0  # Counter to track how many crops have been made

#     # Loop through the image in steps of crop_size (with overlap)
#     for y in range(0, h, crop_size):
#         for x in range(0, w, crop_size):
#             if cropped_count >= 10:  # Stop after 5 crops
#                 break

#             # Define crop boundaries
#             crop_x1, crop_y1 = x, y
#             crop_x2, crop_y2 = min(x + crop_size, w), min(y + crop_size, h)

#             # Extract the crop from the image (with padding if needed)
#             crop = image[crop_y1:crop_y2, crop_x1:crop_x2]
#             if crop.shape[0] < crop_size or crop.shape[1] < crop_size:
#                 padded_crop = np.zeros((crop_size, crop_size, 3), dtype=np.uint8)
#                 padded_crop[:crop.shape[0], :crop.shape[1]] = crop
#                 crop = padded_crop

#             # Check if the crop has more than 10% black area
#             black_pixels = np.sum(crop == 0)
#             total_pixels = crop.size
#             if black_pixels / total_pixels >= 0.1:
#                 continue

#             # Save the cropped image
#             crop_filename = os.path.join(output_image_dir, f'{os.path.splitext(os.path.basename(image_path))[0]}_crop_{cropped_count}.png')
#             cv2.imwrite(crop_filename, crop)

#             # Prepare adjusted labels for this crop
#             crop_labels = []
#             for label in labels:
#                 parts = label.strip().split()
#                 class_index = int(parts[0])
#                 coords = list(map(float, parts[1:]))

#                 # Adjust OBB coordinates relative to the crop
#                 adjusted_coords = []
#                 for i in range(0, len(coords), 2):
#                     x_coord = coords[i] * w
#                     y_coord = coords[i + 1] * h

#                     # Check if the coordinates are within the crop area
#                     if crop_x1 <= x_coord < crop_x2 and crop_y1 <= y_coord < crop_y2:
#                         # Adjust the coordinates relative to the crop
#                         adjusted_coords.append((x_coord - crop_x1) / crop_size)
#                         adjusted_coords.append((y_coord - crop_y1) / crop_size)
#                     else:
#                         # Ensure coordinates stay within the crop boundary
#                         adjusted_coords.append(np.clip((x_coord - crop_x1) / crop_size, 0.0, 1.0))
#                         adjusted_coords.append(np.clip((y_coord - crop_y1) / crop_size, 0.0, 1.0))

#                 if len(adjusted_coords) == 8:
#                     # Check if all 4 points are inside the crop area
#                     coords_check = True
#                     for i in range(0, len(adjusted_coords), 2):
#                         if not (0 <= adjusted_coords[i] <= 1 and 0 <= adjusted_coords[i+1] <= 1):
#                             coords_check = False
#                             break

#                     if not coords_check:
#                         continue

#                     # Ensure the area formed by the 4 points is not zero
#                     quad_points = np.array([(adjusted_coords[i] * crop_size, adjusted_coords[i + 1] * crop_size) 
#                                             for i in range(0, len(adjusted_coords), 2)], dtype=np.int32)
#                     area = cv2.contourArea(quad_points)

#                     if area <= 0:
#                         continue

#                     crop_labels.append(f'{class_index} ' + ' '.join(map(str, adjusted_coords)))

#             # Save the adjusted labels
#             crop_label_filename = os.path.join(output_label_dir, f'{os.path.splitext(os.path.basename(label_path))[0]}_crop_{cropped_count}.txt')
#             with open(crop_label_filename, 'w') as f_out:
#                 if crop_labels:
#                     f_out.write("\n".join(crop_labels) + "\n")
#                 else:
#                     f_out.write("")

#             cropped_count += 1  # Increment the counter


### The code below will ignore labels that lie outside crop region

In [8]:
import os
import cv2
import numpy as np

# Define input and output directories
image_dir = 'val_images'  # Path to your images
label_dir = 'obb_labels'       # Path to your YOLO OBB labels
output_image_dir = 'cropped_images'  # Path to save cropped images
output_label_dir = 'cropped_labels'  # Path to save cropped labels

# Create output directories if they don't exist
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# Define the crop size (1024x1024)
crop_size = 1024

# Get all image and label files
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

# Process the images without progress bar
for image_file, label_file in zip(image_files, label_files):
    image_path = os.path.join(image_dir, image_file)
    label_path = os.path.join(label_dir, label_file)

    # Read the image
    image = cv2.imread(image_path)
    h, w = image.shape[:2]

    # Read the labels
    with open(label_path, 'r') as f:
        labels = f.readlines()

    cropped_count = 0  # Counter to track how many crops have been made

    # Loop through the image in steps of crop_size (with overlap)
    for y in range(0, h, crop_size):
        for x in range(0, w, crop_size):
            if cropped_count >= 10:  # Stop after 10 crops
                break

            # Define crop boundaries
            crop_x1, crop_y1 = x, y
            crop_x2, crop_y2 = min(x + crop_size, w), min(y + crop_size, h)

            # Extract the crop from the image (with padding if needed)
            crop = image[crop_y1:crop_y2, crop_x1:crop_x2]
            if crop.shape[0] < crop_size or crop.shape[1] < crop_size:
                padded_crop = np.zeros((crop_size, crop_size, 3), dtype=np.uint8)
                padded_crop[:crop.shape[0], :crop.shape[1]] = crop
                crop = padded_crop

            # Check if the crop has more than 10% black area
            black_pixels = np.sum(crop == 0)
            total_pixels = crop.size
            if black_pixels / total_pixels >= 0.1:
                continue

            # Save the cropped image
            crop_filename = os.path.join(output_image_dir, f'{os.path.splitext(os.path.basename(image_path))[0]}_crop_{cropped_count}.png')
            cv2.imwrite(crop_filename, crop)

            # Prepare adjusted labels for this crop
            crop_labels = []
            for label in labels:
                parts = label.strip().split()
                class_index = int(parts[0])
                coords = list(map(float, parts[1:]))

                # Adjust OBB coordinates relative to the crop
                adjusted_coords = []
                for i in range(0, len(coords), 2):
                    x_coord = coords[i] * w
                    y_coord = coords[i + 1] * h

                    # Check if the coordinates are within the crop area
                    if crop_x1 <= x_coord < crop_x2 and crop_y1 <= y_coord < crop_y2:
                        # Adjust the coordinates relative to the crop
                        adjusted_coords.append((x_coord - crop_x1) / crop_size)
                        adjusted_coords.append((y_coord - crop_y1) / crop_size)
                    else:
                        # If the point is outside the crop area, discard this label
                        adjusted_coords = []
                        break

                # Only append valid adjusted coordinates if all 4 points are inside the crop area
                if len(adjusted_coords) == 8:
                    # Check if all points are inside the crop boundary
                    coords_check = True
                    for i in range(0, len(adjusted_coords), 2):
                        if not (0 <= adjusted_coords[i] <= 1 and 0 <= adjusted_coords[i+1] <= 1):
                            coords_check = False
                            break

                    if not coords_check:
                        continue

                    # Ensure the area formed by the 4 points is not zero
                    quad_points = np.array([(adjusted_coords[i] * crop_size, adjusted_coords[i + 1] * crop_size) 
                                            for i in range(0, len(adjusted_coords), 2)], dtype=np.int32)
                    area = cv2.contourArea(quad_points)

                    if area <= 0:
                        continue

                    crop_labels.append(f'{class_index} ' + ' '.join(map(str, adjusted_coords)))

            # Save the adjusted labels
            crop_label_filename = os.path.join(output_label_dir, f'{os.path.splitext(os.path.basename(label_path))[0]}_crop_{cropped_count}.txt')
            with open(crop_label_filename, 'w') as f_out:
                if crop_labels:
                    f_out.write("\n".join(crop_labels) + "\n")
                else:
                    f_out.write("")

            cropped_count += 1  # Increment the counter


# Show images with OBB labels 

In [ ]:
import os
import cv2
import numpy as np

# Define input directories and output directory
output_image_dir = 'yolo_dataset/train/images'  # Path where cropped images are saved
output_label_dir = 'yolo_dataset/train/labels'  # Path where cropped labels are saved
output_dir = 'output'  # Path where processed images will be saved

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def draw_obb(image, labels):
    """Draw OBB labels on the image."""
    for label in labels:
        parts = label.strip().split()
        class_index = int(parts[0])
        coords = list(map(float, parts[1:]))

        # Convert normalized coordinates to image coordinates
        h, w = image.shape[:2]
        points = []
        for i in range(0, len(coords), 2):
            x_coord = int(coords[i] * w)
            y_coord = int(coords[i + 1] * h)
            points.append((x_coord, y_coord))

        # Draw the OBB polygon on the image
        points = np.array(points, dtype=np.int32)
        points = points.reshape((-1, 1, 2))
        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)

    return image

def process_and_save_images():
    """Process images in the output folder and save with labels."""
    # Get all image files
    image_files = [f for f in os.listdir(output_image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

    for image_file in image_files:
        image_path = os.path.join(output_image_dir, image_file)
        label_path = os.path.join(output_label_dir, f'{os.path.splitext(image_file)[0]}.txt')

        # Read the image
        image = cv2.imread(image_path)

        # Read the labels
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                labels = f.readlines()

            # Draw labels on the image
            image_with_labels = draw_obb(image.copy(), labels)

            # Define output path
            output_image_path = os.path.join(output_dir, image_file)

            # Save the image with labels
            cv2.imwrite(output_image_path, image_with_labels)

if __name__ == '__main__':
    process_and_save_images()


### Draw square on a image

In [ ]:
from PIL import Image, ImageDraw

# Open the large image
large_image_path = "training_images/P0030.png"  # Replace with your image file path
large_image = Image.open(large_image_path)

# Define the size of the square
square_size = 1024

# Get the dimensions of the large image
width, height = large_image.size

# Calculate how many squares can fit horizontally and vertically
num_squares_x = width // square_size
num_squares_y = height // square_size

# Create a drawing context to draw on the image
draw = ImageDraw.Draw(large_image)

# Draw squares side by side
for i in range(num_squares_x):
    for j in range(num_squares_y):
        # Calculate the top-left corner of the square
        top_left_x = i * square_size
        top_left_y = j * square_size
        bottom_right_x = top_left_x + square_size
        bottom_right_y = top_left_y + square_size
        
        # Draw the rectangle (square) on the image
        draw.rectangle([top_left_x, top_left_y, bottom_right_x, bottom_right_y], outline="red", width=5)  # You can adjust the outline color/width

# Save or display the image with drawn squares
large_image.save("output_image_with_squares.jpg")
large_image.show()  # Optional: Show the image in a viewer
